In [1]:
import math
import json
import sys
import pandas as pd
import rasterio
from rasterio.mask import mask
# from libtiff import TIFF
# import libtiff
import sys
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
import pickle
import h5py
import scipy.misc
import seaborn as sns
import math
import os
import matplotlib.pyplot as plt

In [36]:
originalFolder = 'data/2014_dec'
cutFilesFolder = originalFolder+'_cutFiles'
csvFileName = cutFilesFolder+'.csv'

In [3]:
onlyfiles = [f for f in listdir(cutFilesFolder) if isfile(join(cutFilesFolder, f))]

In [4]:
village_File_Dictionary={}
for tempStr in onlyfiles:
    splitUpList=tempStr[:-4].split('@')
    splitUpList=splitUpList[1:]
    keyVillage = '@'.join(splitUpList)
    if (not (keyVillage in village_File_Dictionary)):
        village_File_Dictionary[keyVillage]=[tempStr]
    else:
        village_File_Dictionary[keyVillage].append(tempStr)

In [5]:
print(len(onlyfiles))
print(len(village_File_Dictionary))

199185
197851


In [42]:
# 0.2 from -10 to -8
# 0.1 from -8 to 3 
# 0.2 from 3 to 5
range1 = list(range(-100,-80,2))
range2 = list(range(-80,30,1))
range3 = list(range(30,50,2))
(range1.extend(range2))
range1.extend(range3)
rangeBins = [t/10 for t in range1]
# print(rangeBins)

columns1=['vCode2001','vid','vCode2011']
col_help=['light_'+str(t) for t in range(129)]
columns1.extend(col_help)
binningDf = pd.DataFrame(columns=columns1)

village_bins_dict={}
numVillageC = 0
for keyVillage,villageFileList in village_File_Dictionary.items():
    numVillageC = numVillageC + 1
    currVillage_pixels= None
    for cVillageFile in villageFileList:
        #print(cVillageFile)
        im = Image.open(cutFilesFolder+'/'+cVillageFile) 
        imarray = np.array(im)
        flattenData=imarray.flatten()
        flattenData=flattenData[flattenData != 0]
        if(currVillage_pixels is None):
            currVillage_pixels = flattenData
        else:
            currVillage_pixels = np.append(currVillage_pixels,flattenData)
    currVillage_pixels=np.log(currVillage_pixels)
    currVillage_pixels = currVillage_pixels[~np.isnan(currVillage_pixels)]
    currVillage_pixels[currVillage_pixels<(-10.0)] = (-10.0)
    currVillage_pixels[currVillage_pixels>4.8] = (4.8)
    binning=np.histogram(currVillage_pixels, bins=rangeBins)
    list_row=(keyVillage.split('@'))
    list_row.extend(binning[0])
    binningDf.loc[numVillageC-1] =  list_row
    
binningDf.to_csv(csvFileName)

/home/satbigvm/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in log


500


In [35]:
for key, hist in village_bins_dict.items():
    print(hist)
    print(len(hist))
    break

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0   58  161
  108  896 1192  743  462  630  361  225  240  241  636    0   75  335
   88  350  240    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
129
